In [179]:
using JLD2
using Flux
using MLJ
using MLJLIBSVMInterface
using StatisticalMeasures
using CategoricalArrays
MLJ.@load KNNClassifier pkg=NearestNeighborModels

include("../src/filters.jl")
include("../src/reduction.jl")

@JLD2.load "variables/train_dataset.jld2" train_inputs train_targets
@JLD2.load "variables/test_dataset.jld2" test_inputs test_targets
@JLD2.load "variables/cross_val_index.jdl2" cross_val_index


import NearestNeighborModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\Usuario\.julia\packages\MLJModels\ziReN\src\loading.jl:159


1-element Vector{Symbol}:
 :cross_val_index

### Constructor de arquitectura MLP

In [180]:
function buildANN(n_inputs::Int, hidden_layers::Vector{Int}, n_outputs::Int; activation = relu)
    layers = Any[]

    # capas ocultas
    in_dim = n_inputs
    for h in hidden_layers
        push!(layers, Dense(in_dim, h, activation))
        in_dim = h
    end

    # capa de salida
    push!(layers, Dense(in_dim, n_outputs))
    push!(layers, Flux.softmax)

    return Chain(layers...)
end

buildANN (generic function with 1 method)

### Entrenamiento de dicho MLP

In [181]:
function trainANN(ann::Chain, dataset::Tuple{DataFrame, BitArray};
    epochs::Int=1000, learningRate::Real=0.001, minLossChange::Real=1e-7, lossChangeWindowSize::Int=5)

    inputs = Matrix(dataset[1]) |> x -> Float32.(x)
    targets = Matrix(dataset[2]) |> x -> Float32.(x)
    # Flux espera columnas = muestras
    inputs = inputs'
    targets = targets'

    loss(model, x, y) = Flux.Losses.crossentropy(model(x), y);
    trainingLosses = Float32[loss(ann, inputs, targets)];
    opt_state = Flux.setup(Adam(learningRate), ann);

    for epoch in 1:epochs
        Flux.train!(loss, ann, [(inputs, targets)], opt_state);
        current_loss = Float32(loss(ann, inputs, targets));
        push!(trainingLosses, current_loss);
    end;

    # Predicciones
    ŷ = ann(inputs) 
    y_pred = Flux.onecold(ŷ)           # Covertimos a One-hot-encoding
    y_true = Flux.onecold(targets)

    # A formato MLJ
    y_pred_cat = categorical(y_pred)
    y_true_cat = categorical(y_true)

    cm = confusion_matrix(y_true_cat, y_pred_cat)

    # Sensitivity (recall) de la clase 1
    sens = MLJ.sensitivity(cm)

    # Accuracy
    acc = MLJ.accuracy(cm)

    return (
        losses = trainingLosses,
        accuracy = acc,
        sensitivity = sens
    )

end;

### Creación y entrenamiento de SVM

In [ ]:
function createSVM(dataset::Tuple{DataFrame, BitArray}, kernel::String, C::Real; gamma::Real=1.0, degree::Int=3, coef0::Real=0.0)

    # deshacemos One-Hot-Encoding 
    inputs, labels = dataset; 
    targets = [sum(findall(row)) for row in eachrow(labels)]

    if kernel == "linear"
        k = LIBSVM.Kernel.Linear
    elseif kernel == "rbf"
        k = LIBSVM.Kernel.RadialBasis
    elseif kernel == "poly"
        k = LIBSVM.Kernel.Polynomial
    elseif kernel == "sigmoid"
        k = LIBSVM.Kernel.Sigmoid
    else
        error("Kernel no soportado: $kernel")
    end

    model = SVMClassifier(
        kernel = k,
        cost = Float64(C),
        gamma = Float64(gamma),
        degree = Int32(degree),
        coef0 = Float64(coef0)
    )

    mach = machine(
        model,
        inputs,
        categorical(targets)
    )

    MLJ.fit!(mach)

    # Predicciones sobre el mismo conjunto
    y_pred = MLJ.predict(mach, inputs)
    y_pred_labels = mode.(y_pred)           # Extrae los labels
    y_pred_vec = Int.(y_pred_labels)     # Vector{Int}
    y_pred_cat = categorical(y_pred_vec)    # Vector categórico

    y_true_cat = categorical(targets)

    # Métricas
    cm = confusion_matrix(y_true_cat, y_pred_cat)
    acc = accuracy(cm)
    sens = sensitivity(cm)


    cm = confusion_matrix(y_true_cat, y_pred_cat)
    acc = accuracy(cm)
    sens = sensitivity(cm)
    
    return mach, acc, sens
end

createSVM (generic function with 2 methods)

#### Creación y entrenamiento de KNN

In [ ]:
function createKNN(dataset::Tuple{DataFrame, AbstractMatrix}; k::Int = 5)

    # Deshacemos One-Hot-Encoding   
    inputs, labels = dataset; 
    targets = [sum(findall(row)) for row in eachrow(labels)]

    model = NearestNeighborModels.KNNClassifier(K = k)

    mach = machine(
        model,
        inputs,
        categorical(targets)
    )

    MLJ.fit!(mach)
    # Predicciones sobre el mismo conjunto
    y_pred = MLJ.predict(mach, inputs)
    y_pred_labels = mode.(y_pred)           # Extrae los labels
    y_pred_vec = Int.(y_pred_labels)     # Vector{Int}
    y_pred_cat = categorical(y_pred_vec)    # Vector categórico

    y_true_cat = categorical(targets)

    # Métricas
    cm = confusion_matrix(y_true_cat, y_pred_cat)
    acc = accuracy(cm)
    sens = sensitivity(cm)


    cm = confusion_matrix(y_true_cat, y_pred_cat)
    acc = accuracy(cm)
    sens = sensitivity(cm)
    
    return mach, acc, sens
end

createKNN (generic function with 4 methods)

## Entrenamiento de clasificadores con filtrado y reducción (sin Cross Validation)

##### PCA + ANOVA + ANN ([50])

In [186]:
inputs, targets = train_inputs, train_targets

# 1. PCA
X_pca, pca_model = pca(inputs, 0.95)

# 2. ANOVA
anova_scores = anova((X_pca, train_targets));
anova_index = findall(x -> x >= 0.3, anova_scores);
new_dataset = (X_pca[:, anova_index], train_targets);

# 3. ANN
n_inputs = size(new_dataset[1], 2)
n_outputs = size(new_dataset[2], 2)
ann_model = buildANN(n_inputs, [50], n_outputs)
losses, accuracy, sensitivity = trainANN(ann_model, (new_dataset[1], new_dataset[2]))

println(accuracy)
println(sensitivity)



0.7785830178474852
0.748650732459522


┌ Warning: Levels not explicitly ordered. Using the order CategoricalValue{Int64, UInt32}[1, 2, 3, 4, 5, 6]. 
└ @ StatisticalMeasures.ConfusionMatrices C:\Users\Usuario\.julia\packages\StatisticalMeasures\UTtxb\src\confusion_matrices.jl:339


##### PEARSON + LDA + SVM (c=0.5)

In [187]:
# 1. PEARSON
filtered_train_inputs, correlations = pearson((train_inputs, train_targets))
new_dataset = (filtered_train_inputs, train_targets);

# 2. LDA
X_proj, W = lda(new_dataset)
X_proj_real = real.(X_proj)      # si hay complejos, toma la parte real
X_proj_clean = coalesce.(X_proj_real, 0.0)   # rellena posibles missing con 0
X_lda = DataFrame(X_proj_clean, :auto)
new_dataset = (X_lda, train_targets)

# 3. SVM
mach = createSVM(new_dataset, "linear", 0.5)


┌ Info: Training machine(SVC(kernel = Linear, …), …).
└ @ MLJBase C:\Users\Usuario\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


trained Machine; caches model-specific representations of data
  model: SVC(kernel = Linear, …)
  args: 
    1:	Source @812 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @258 ⏎ AbstractVector{Multiclass{6}}


##### RFE + (sin reducción de dimensionalidad) + KNN(k=20)

In [188]:
# 1. RFE 
filtered_features = rfe_logistic((train_inputs, train_targets));
filtered_features_names = names(train_inputs)[filtered_features]

# eliminamos valores faltantes
X_clean = DataFrame(
    coalesce.(Matrix(train_inputs[:, filtered_features_names]), 0.0),
    names(train_inputs[:, filtered_features_names])
)

new_dataset = (X_clean, train_targets);

# Sin reducción de dimensionalidad

# 2. KNN
mach = createKNN(new_dataset; k=20)

┌ Info: Training machine(KNNClassifier(K = 20, …), …).
└ @ MLJBase C:\Users\Usuario\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


trained Machine; caches model-specific representations of data
  model: KNNClassifier(K = 20, …)
  args: 
    1:	Source @993 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @876 ⏎ AbstractVector{Multiclass{6}}


##### KENDALL + ICA + KNN(k=10)

In [ ]:
# 1. KENDALL
kendall_coeff = kendall((train_inputs, train_targets))
kendall_index = findall(x -> x >= 0.3, kendall_coeff);
new_dataset = (train_inputs[:, kendall_index], train_targets);

# 2. ICA
S, W = fastica(new_dataset; tol=0.01, max_iter=100)
S = DataFrame(S, :auto)
new_dataset = (S, train_targets)

# 3. KNN
mach = createKNN(new_dataset; k=10)

┌ Info: Training machine(KNNClassifier(K = 10, …), …).
└ @ MLJBase C:\Users\Usuario\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


trained Machine; caches model-specific representations of data
  model: KNNClassifier(K = 10, …)
  args: 
    1:	Source @912 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @325 ⏎ AbstractVector{Multiclass{6}}


##### SPEARMAN + LDA + ANN([100, 50])

In [86]:
# 1. SPEARMAN
spearman_scores = spearman((train_inputs, train_targets));
spearman_index = findall(x -> x >= 0.01, spearman_scores);
new_dataset = (train_inputs[:, spearman_index], train_targets);

# 2. LDA
X_proj, W = lda(new_dataset)
X_proj_real = real.(X_proj)      # si hay complejos, toma la parte real
X_proj_clean = coalesce.(X_proj_real, 0.0)   # rellena posibles missing con 0
X_lda = DataFrame(X_proj_clean, :auto)
new_dataset = (X_lda, train_targets)

# 3. ANN
n_inputs = size(new_dataset[1], 2)
n_outputs = size(new_dataset[2], 2)
ann_model = buildANN(n_inputs, [100,50], n_outputs)
losses = trainANN(ann_model, (new_dataset[1], new_dataset[2]))

1001-element Vector{Float32}:
 1.7307363
 1.7010912
 1.6721834
 1.6442572
 1.6174283
 1.5916183
 1.5664014
 1.5416322
 1.5173337
 1.4934605
 ⋮
 0.23180948
 0.23177567
 0.23173955
 0.23171045
 0.23167728
 0.23164198
 0.23160861
 0.23157687
 0.23154181

##### (sin filtrado) + PCA + SVM(c=1.0)

In [ ]:
# 1. PCA
X_pca, pca_model = pca(inputs, 0.95)

# 2. SVM
mach = createSVM(new_dataset, "linear", 1.0)


┌ Info: Training machine(SVC(kernel = Linear, …), …).
└ @ MLJBase C:\Users\Usuario\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


trained Machine; caches model-specific representations of data
  model: SVC(kernel = Linear, …)
  args: 
    1:	Source @108 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @888 ⏎ AbstractVector{Multiclass{6}}


##### PEARSON + LDA + ANN([100])

In [87]:
# 1. PEARSON 
filtered_train_inputs, correlations = pearson((train_inputs, train_targets), threshold=0.1);
new_dataset = (filtered_train_inputs, train_targets);

# 2. LDA
X_proj, W = lda(new_dataset)
X_proj_real = real.(X_proj)      # si hay complejos, toma la parte real
X_proj_clean = coalesce.(X_proj_real, 0.0)   # rellena posibles missing con 0
X_lda = DataFrame(X_proj_clean, :auto)
new_dataset = (X_lda, train_targets)

# 3. ANN
n_inputs = size(new_dataset[1], 2)
n_outputs = size(new_dataset[2], 2)
ann_model = buildANN(n_inputs, [100], n_outputs)
losses = trainANN(ann_model, (new_dataset[1], new_dataset[2]))

1001-element Vector{Float32}:
 1.7635249
 1.7449391
 1.7266634
 1.7086916
 1.6910176
 1.6736385
 1.6565499
 1.6397454
 1.6232218
 1.6069708
 ⋮
 0.313009
 0.3129705
 0.31293198
 0.31289357
 0.31285518
 0.31281686
 0.3127786
 0.3127404
 0.31270224

##### KENDALL + (sin reducción) + SVM(c=0.1)

In [90]:
# 1. KENDALL
kendall_coeff = kendall((train_inputs, train_targets))
kendall_index = findall(x -> x >= 0.3, kendall_coeff);

# Eliminamos datos missing
X_clean = DataFrame(
    coalesce.(Matrix(train_inputs[:, kendall_index]), 0.0),
    names(train_inputs[:, kendall_index])
)

new_dataset = (X_clean, train_targets);

# 2. SVM
mach = createSVM(new_dataset, "linear", 0.1)

┌ Info: Training machine(SVC(kernel = Linear, …), …).
└ @ MLJBase C:\Users\Usuario\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


trained Machine; caches model-specific representations of data
  model: SVC(kernel = Linear, …)
  args: 
    1:	Source @160 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @715 ⏎ AbstractVector{Multiclass{6}}


#### (sin filtrado) + LDA + KNN(k=1)

In [89]:
# 1. LDA
X_proj, W = lda(new_dataset)
X_proj_real = real.(X_proj)      # si hay complejos, toma la parte real
X_proj_clean = coalesce.(X_proj_real, 0.0)   # rellena posibles missing con 0
X_lda = DataFrame(X_proj_clean, :auto)
new_dataset = (X_lda, train_targets)

# 2. KNN
mach = createKNN(new_dataset; k=1)

┌ Info: Training machine(KNNClassifier(K = 1, …), …).
└ @ MLJBase C:\Users\Usuario\.julia\packages\MLJBase\7nGJF\src\machines.jl:499


trained Machine; caches model-specific representations of data
  model: KNNClassifier(K = 1, …)
  args: 
    1:	Source @291 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @785 ⏎ AbstractVector{Multiclass{6}}


## Entrenamiento de clasificadores con filtrado y reducción (ahora con Cross Validation)

In [ ]:
k = maximum(cross_val_index)
accuracies = Float64[]
sensitivities = Float64[]

for fold in 1:k
    println("Fold $fold")

    train_idx = findall(x -> x != fold, cross_val_index)
    test_idx  = findall(x -> x == fold, cross_val_index)

    X_train = inputs[train_idx, :]
    y_train = targets[train_idx, :]

    X_test  = inputs[test_idx, :]
    y_test  = targets[test_idx, :]

    # 2. PCA (solo con train)
    X_train_pca, pca_model = pca(X_train, 0.95)
    X_test_pca = pca(X_test, 0.95)

    # 3. ANOVA (solo con train)
    anova_scores = anova((X_train_pca, y_train))
    selected_idx = findall(x -> x >= 0.3, anova_scores)

    X_train_sel = X_train_pca[:, selected_idx]
    X_test_sel  = X_test_pca[:, selected_idx]

    # 4. ANN
    n_inputs = size(X_train_sel, 2)
    n_outputs = size(y_train, 2)

    ann_model = buildANN(n_inputs, [50], n_outputs)

    losses, acc, sens = trainANN(ann_model, (X_train_sel, y_train), 
                                 (X_test_sel, y_test))

    push!(accuracies, acc)
    push!(sensitivities, sens)

    println("Fold $fold → accuracy = $acc, sensitivity = $sens")
end

println("\n=== RESULTADOS FINALES ===")
println("Accuracy promedio: ", mean(accuracies))
println("Sensitivity promedio: ", mean(sensitivities))


Fold 1

MethodError: MethodError: no method matching trainANN(::Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, typeof(softmax)}}, ::Tuple{DataFrame, BitMatrix}, ::Tuple{DataFrame, BitMatrix})
The function `trainANN` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  trainANN(::Chain, ::Tuple{DataFrame, BitArray}; epochs, learningRate, minLossChange, lossChangeWindowSize)
   @ Main c:\Users\Usuario\Documents\adrispul\Universidad\INTELIGENCIA ARTIFICIAL\3º\1º SEMESTRE\MAAA\PRÁCTICAS\GitHub_Repository\Practicas_Julia\Practica4\notebooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W2sZmlsZQ==.jl:1
